In [1]:
!pip install python-dotenv

You should consider upgrading via the 'C:\Users\franz\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [1]:
import os

import dotenv
dotenv.load_dotenv()

import sys
sys.path.append('..')
import temimbo

import importlib
importlib.reload(temimbo)

<module 'temimbo' from 'c:\\Users\\franz\\OneDrive\\Desktop\\BA\\BA\\LanguageTeacher\\notebooks\\..\\temimbo\\__init__.py'>

In [2]:
connector_openai = temimbo.ConnectorOpenAI(
    openai_key = os.getenv('OPENAI_KEY')
)
tg = temimbo.TaskGenerator(connector_llm = connector_openai)
evaluator = temimbo.AnswerEvaluator(connector_llm = connector_openai)


ui = temimbo.UserInterface()
db = temimbo.DatabaseClientLocalFile(connection_string='./database')
formater = temimbo.Formater()



In [3]:
profile = await db.load_profile(id='Klaus')
domain = await ui.choose_domain()
task_type = await ui.choose_task_type()
level, training_goals_subset = await tg.incorporate_profile_in_task(profile, domain)

In [4]:
prompt = await tg.generate_prompt(
    level = level,
    training_goals_subset = training_goals_subset,
    domain = domain,
    task_type = task_type,
)
print(prompt)

Pretend you are a academic english teacher. Generate only the task itself, no other text.
Generate a single choice (4 possible answer choices per task), only one answer is correct task, in the area of vocabulary teaching, targeting on Cars and Mars.
Do not provide the correct answers, just the task output. Only one task. It should be for academic english learners. Therefore, the tasks have to train academic english!
The following is an example of how the task should be generated. The language level should be C1 or higher

Example text ____ with gap
a) answer option 1
b) answer option 2
c) answer  option 3
d) answer  option 4


In [5]:
raw_output = await tg.generate_task(prompt)
print(raw_output)

Mars is a planet ____ with the Earth, as they both have an atmosphere and seasons.
a) densely populated
b) in direct competition
c) closely associated
d) highly susceptible


In [6]:
formatted_output_task = await formater.output_task_formatting(raw_output)
user_answer = await ui.answer_task(formatted_output_task)
formated_user_answer = await formater.learner_answer_formatting(user_answer)

prompt = await evaluator.generate_prompt(
    domain = domain,
    formatted_output_task = formatted_output_task,
    formated_user_answer = formated_user_answer,
    task_type = task_type,
)
print(prompt)


Pretend you are an academic english teacher. You have three tasks:
Evaluate the following answer of a student to a given task with boolean values.
Name the topic the student has to practice more.
Give sensible feedback to the student. Tell them what is wrong, and what they have to practice.

This was the given task:
Hello user, you look pretty today. Now Answer this.

Mars is a planet ____ with the Earth, as they both have an atmosphere and seasons.
a) densely populated
b) in direct competition
c) closely associated
d) highly susceptible

This was the student's answer: 
I choose A


In [7]:
NL_feedback, correctness, training_goals = await evaluator.evaluate_learner_answer(prompt)
print('NL_feedback =', NL_feedback)
print('correctness =', correctness)
print('training_goals =', training_goals)

NL_feedback = Oh my darling, your mistakes are comprehensible, but...
correctness = False
training_goals = vocabulary_goals=['Cat anatomical parts'] grammar_goals=["Use of don't", 'Simple past, even though pasts are never simple'] text_goals=[]


In [8]:
new_profile = await evaluator.update_learner_profile(training_goals, profile)
print(f'Old profile:\n{profile}\n\nNew profile:\n{new_profile}')

Old profile:
name='Klaus' level=Level(vocabulary_level=4, grammar_level=4, text_level=2) training_goals=TrainingGoals(vocabulary_goals=['Cars', 'Mars'], grammar_goals=[], text_goals=['Talk about powerful cars'])

New profile:
name='Klaus' level=Level(vocabulary_level=4, grammar_level=4, text_level=2) training_goals=TrainingGoals(vocabulary_goals=['Cars', 'Mars', 'Cat anatomical parts'], grammar_goals=["Use of don't", 'Simple past, even though pasts are never simple'], text_goals=['Talk about powerful cars'])


In [9]:
# This overwrites the original profile
# For testing purposes, it's easier to just comment it out
# await db.save_profile(new_profile)